# Scraping first half of data

In [ ]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [116]:
#Initialize quotes list and prepare to scrape first page
quotes = []
authors = []
likes = []

#list of tags, and so indirectly, list of web pages to be scraped. This is only half the full list; it was broken up
# to make troubleshooting easier
tag_list = ['love', 'life', 'inspirational', 'humor', 'philosophy', 'god', 'inspirational-quotes', 'truth', 
            'wisdom', 'poetry', 'romance', 'death', 'happiness', 'hope', 'faith', 'inspiration', 'quotes']

# for every tag, 100 pages of quotes are provided by goodreads.com. Here we scrape data from all of them.
for i in tag_list:
    for j in range(100):
        url = "https://www.goodreads.com/quotes/tag/{}?page={}".format(i, j+1)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        # this boolean list will be used to handle errors that arise when webscraping
        success = [True for k in range(len(soup.find_all(class_="quoteText")))]
        
        #initialize a count
        count_1 = -1
        
        #for every quote found, add it to the list of quotes
        for my_tag in soup.find_all(class_="quoteText"):
            count_1 += 1
            try:
                # Occasionally, an issue would arise with scraping author data. If this happens, none of the data 
                # associated with that quote should be added to the dataset. To accomplish this, in the event of 
                # an error, appending the author data fails, appending the quotes data is skipped, and the success 
                # boolean at the current index is set to false.
                authors.append(my_tag.text.split("//")[0].split("―")[1].split(",")[0])
                quotes.append(my_tag.text.split("//")[0].split("―")[0])
            except:
                success[count_1] = False
                continue
        
        #initialize a count that will mirror the values of the first count as the following loop is executed.
        count_2 = -1
        
        # The likes data is scraped from different html code, so it is scraped in a separate for loop. A count is 
        # in place to ensure that certain rows are skipped when the corresponding data was not successfully scraped.
        for my_tag in soup.find_all('a', {'class': 'smallText', 'title': 'View this quote'}):
            count_2 += 1
            # if the corresponding author and quote data was successfully scraped
            if success[count_2] == True:
                likes.append(my_tag.text.split(" ")[0])
        
        #way to monitor progress
        if (j+1) % 100 == 0:    
            print('tag done')

tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done


In [117]:
#these should all be the same magnitude
print(len(quotes))
print(len(authors))
print(len(likes))

50995
50995
50995


In [119]:
#get rid of extra white spaces
quotes = [quote.strip() for quote in quotes]
authors = [author.strip() for author in authors]
likes = [like.strip() for like in likes]

In [120]:
#get rid of parentheses around quotes
quotes = [quote[1:] for quote in quotes]
quotes = [quote[:-1] for quote in quotes]

# Scraping second half of data

The code in this section is analogous to the code in the first section

In [121]:
quotes_2 = []
authors_2 = []
likes_2 = []

tag_list_2 = ['writing', 'religion', 'life-lessons', 'motivational', 'success', 'relationships', 'spirituality', 
            'time', 'love-quotes', 'knowledge', 'life-quotes', 'science', 'books']

for i in tag_list_2:
    for j in range(100):
        url = "https://www.goodreads.com/quotes/tag/{}?page={}".format(i, j+1)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')

        success = [True for k in range(len(soup.find_all(class_="quoteText")))]
        
        count_1 = -1
        #for every quote found, add it to the list of quotes
        for my_tag in soup.find_all(class_="quoteText"):
            count_1 += 1
            try:
                authors_2.append(my_tag.text.split("//")[0].split("―")[1].split(",")[0])
                quotes_2.append(my_tag.text.split("//")[0].split("―")[0])
            except:
                success[count_1] = False
                continue
        
        count_2 = -1
        for my_tag in soup.find_all('a', {'class': 'smallText', 'title': 'View this quote'}):
            count_2 += 1
            if success[count_2] == True:
                    likes_2.append(my_tag.text.split(" ")[0])

        if (j+1) % 100 == 0:    
            print('tag done')

tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done
tag done


In [122]:
quotes_spare_2 = quotes_2
authors_spare_2 = authors_2
likes_spare_2 = likes_2

In [123]:
quotes_2 = [quote_2.strip() for quote_2 in quotes_2]
authors_2 = [author_2.strip() for author_2 in authors_2]
likes_2 = [like_2.strip() for like_2 in likes_2]

In [124]:
quotes_2 = [quote_2[1:] for quote_2 in quotes_2]
quotes_2 = [quote_2[:-1] for quote_2 in quotes_2]

# Combining the data and creating dataframe

In [134]:
#Combining data
Quotes = quotes + quotes_2
Authors = authors + authors_2
Likes = likes + likes_2

In [135]:
#Create dataframe with quotes
df = pd.DataFrame(Quotes,columns=['Quote'])

In [136]:
df['Author'] = Authors

In [137]:
df['Likes'] = Likes

In [138]:
df.shape

(89995, 3)

In [142]:
df = df.drop_duplicates(subset=['Quote'])

In [143]:
df = df.dropna()

In [144]:
df.shape

(70773, 3)

In [22]:
df = pd.read_csv('quote_data.csv').drop(columns=['Unnamed: 0'])

In [23]:
df.head()

,Quote,Likes
0,"I'm selfish, impatient and a little insecure. ...",153436
1,You've gotta dance like there's nobody watchin...,120542
2,You know you're in love when you can't fall as...,117305
3,A friend is someone who knows all about you an...,79638
4,Darkness cannot drive out darkness: only light...,73740


In [17]:
df.to_csv('quote_data.csv')